In [1]:
music_type = 'chorales'
dataset = 'datasets/' + music_type

In [2]:
import os
import util

In [3]:
'''
files = [f for f in os.listdir(dataset+'/krn/') if f.endswith('.krn')]

for file in files[:]:
    basename = file[:-4]
    ! transpose -k c $dataset/krn/$basename\.krn | rchoraledata > $dataset/dat/$basename\.dat
'''

"\nfiles = [f for f in os.listdir(dataset+'/krn/') if f.endswith('.krn')]\n\nfor file in files[:]:\n    basename = file[:-4]\n    ! transpose -k c $dataset/krn/$basename\\.krn | rchoraledata > $dataset/dat/$basename\\.dat\n"

In [4]:
files = [f for f in os.listdir(dataset+'/dat/') if f.endswith('.dat')]
files.sort()

In [5]:
# c_data: [song, measure, chord, pitch]
# t_data: [song, measure, dur]
t_data, c_data, t_val2idx, t_idx2val, c_val2idx, c_idx2val = (util.parse_dat_dir(dataset, files))

In [6]:
test = util.gen_data_rec(c_data, c_val2idx)

In [7]:
idx_name = [c_idx2val[i] for i in range(len(c_val2idx))]
co_mat = util.gen_co_mat(c_data, c_val2idx)
import pandas as pd
df_chord = pd.DataFrame(co_mat, index=idx_name)

In [8]:
dim = 80
principal_data = util.pca_wrapper(co_mat, dim)
col_name = ['component_' + str(i) for i in range(dim)]

In [9]:
df_chord_principal = pd.DataFrame(data=principal_data, columns=col_name, index=idx_name)

In [10]:
emb, names = util.gen_chord_emb(c_data, principal_data, c_val2idx)

In [13]:
from sklearn.manifold import TSNE
C_embedded_on = TSNE(n_components=2).fit_transform(emb)

In [14]:
import matplotlib.pyplot as plt
%matplotlib agg

In [18]:
fig = plt.figure(figsize=(200, 200))
for i, key in enumerate(names):
    plt.text(C_embedded_on[i,0], C_embedded_on[i,1], util.idx_tuple_to_chord_name(key, c_idx2val, util.get_note_name_by_idx))
    plt.scatter(C_embedded_on[i,0], C_embedded_on[i,1], marker='h', s=200, color='grey', alpha=0.4)
plt.savefig('graphs/'+music_type+'/chord_name.svg')

In [19]:
fig = plt.figure(figsize=(200, 200))
for i, key in enumerate(names):
    plt.text(C_embedded_on[i,0], C_embedded_on[i,1], util.idx_tuple_to_chord_name(key, c_idx2val, util.get_deg_7_by_idx))
    plt.scatter(C_embedded_on[i,0], C_embedded_on[i,1], marker='h', s=200, color='grey', alpha=0.4)
plt.savefig('graphs/'+music_type+'/7_deg.svg')

In [12]:
import numpy as np
np.savez_compressed(dataset+'/emb/'+'emb', emb=emb)

In [17]:
def flatten_by_song(data):
    # can be used to chord2vec
    # flatten_data: [song, item], item can be chord or time duration
    ret = []
    for song in data[:]:
        temp = []
        for measure in song:
            temp.extend(measure)
        ret.append(temp)
    return ret

In [18]:
flatten_data = flatten_by_song(util.gen_data_rec(c_data, c_val2idx))